In [50]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks

In [51]:
df = pd.read_csv("/content/synthetic_transactions.csv")


In [54]:
# Drop non-numeric columns
df_processed = df.drop(columns=["transaction_id", "sender_address", "receiver_address"])

In [55]:
df_processed.head()

,amount,timestamp,gas_fee,transaction_count,wallet_age,is_fraud
0,1.0374,1740657035,0.033979,3,269,0
1,21.9717,1721771081,0.078390,225,96,0
2,48.6431,1732206524,0.067293,138,201,0
3,68.9254,1736983993,0.016187,375,44,0
4,96.2830,1725899748,0.025305,91,306,1


In [56]:
# Convert timestamp to datetime and extract features
df_processed["timestamp"] = pd.to_datetime(df_processed["timestamp"], unit="s")
df_processed["hour"] = df_processed["timestamp"].dt.hour
df_processed["day_of_week"] = df_processed["timestamp"].dt.dayofweek

In [57]:
# Separate features and target
X = df_processed.drop(columns=["is_fraud"])
y = df_processed["is_fraud"]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=0.3,stratify=y)

print(X_train.shape)
print(X_test.shape)

(3500, 7)
(1500, 7)


In [59]:
# Feature selection
import numpy as np
from sklearn.preprocessing import MinMaxScaler

columns = ['amount', 'gas_fee', 'transaction_count', 'wallet_age', 'hour', 'day_of_week']

scaler = MinMaxScaler()
#Drop the timestamp column before scaling
X_train = X_train.drop(columns=['timestamp'])
X_test = X_test.drop(columns=['timestamp'])


for c in columns:
  X_train[c] = X_train[c].apply(lambda x: np.log(x) if x > 0 else 0)
  X_test[c] = X_test[c].apply(lambda x: np.log(x) if x > 0 else 0)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
np.isnan(X_train).sum()

0

In [61]:

def create_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train model
model = create_model()
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Save model
model.save("mlp_fraud_detection.keras")
print("Model saved successfully!")


Epoch 1/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.5437 - loss: 1.0129 - val_accuracy: 0.5013 - val_loss: 0.8242
Epoch 2/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6159 - loss: 0.8558 - val_accuracy: 0.6873 - val_loss: 0.7930
Epoch 3/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6625 - loss: 0.7678 - val_accuracy: 0.7433 - val_loss: 0.7535
Epoch 4/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6869 - loss: 0.7217 - val_accuracy: 0.7560 - val_loss: 0.7145
Epoch 5/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6933 - loss: 0.7022 - val_accuracy: 0.7547 - val_loss: 0.6785
Epoch 6/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7066 - loss: 0.6837 - val_accuracy: 0.7513 - val_loss: 0.6475
Epoch 7/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7423 - loss: 0.6467 - val_accuracy: 0.7747 - val_loss: 0.6149
Epoch 8/100
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7413 - loss: 0.6477 - val_accuracy: 0.7

In [62]:
model = create_model()
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

history = model.fit(
    X_train, y_train,
    epochs=200, batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")

# Save model
model.save("mlp_fraud_detection.keras")
print("Model saved successfully!")


Epoch 1/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5414 - loss: 1.0508 - val_accuracy: 0.6413 - val_loss: 0.7957 - learning_rate: 5.0000e-04
Epoch 2/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6266 - loss: 0.8576 - val_accuracy: 0.6413 - val_loss: 0.7705 - learning_rate: 5.0000e-04
Epoch 3/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6654 - loss: 0.7864 - val_accuracy: 0.6413 - val_loss: 0.7460 - learning_rate: 5.0000e-04
Epoch 4/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6932 - loss: 0.7252 - val_accuracy: 0.6420 - val_loss: 0.7206 - learning_rate: 5.0000e-04
Epoch 5/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7067 - loss: 0.7034 - val_accuracy: 0.6493 - val_loss: 0.6957 - learning_rate: 5.0000e-04
Epoch 6/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7525 - loss: 0.6581 - val_accuracy: 0.7027 - val_loss: 0.6661 - learning_rate: 5.0000e-04
Epoch 7/200
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7340